# 📊 REGRESSION - COMPLETE IMPLEMENTATION

## 🏗️ Prediksi Kekuatan Beton (Concrete Compressive Strength)

**© Muhammad Ketsar Ali Abi Wahid**

---

## 📚 Learning Objectives

Setelah menyelesaikan notebook ini, Anda akan mampu:

✅ Memahami complete end-to-end regression pipeline (10 FASE)

✅ Mengimplementasikan 12 regression algorithms

✅ Melakukan hyperparameter tuning (Grid Search, Random Search, Optuna)

✅ Mengevaluasi model dengan multiple metrics

✅ Menginterpretasi model dengan SHAP

✅ Membuat model report

---

## ⏱️ Estimated Time: 6-8 hours

---

## 📦 Requirements

```
- numpy
- pandas
- matplotlib
- seaborn
- scikit-learn
- xgboost
- lightgbm
- catboost
- optuna
- shap
```

---

# 🔷 FASE 1: DATA LOADING & INITIAL EXPLORATION

**Analogi**: Seperti saat Anda pertama kali membuka kotak berisi puzzle. Anda hitung ada berapa pieces, lihat ukuran kotak, cek apakah ada yang rusak.

**Tujuan Fase Ini:**
- Load dataset
- Check shape (jumlah baris dan kolom)
- Check data types
- Check memory usage
- Initial observations

## 1.1 Setup & Import Libraries

**Penjelasan**: Kita import semua library yang dibutuhkan di awal.

**Analogi**: Seperti menyiapkan semua peralatan masak sebelum mulai memasak.

In [ ]:
# ========================================
# IMPORT LIBRARIES
# ========================================

# Data manipulation
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation

# Visualization
import matplotlib.pyplot as plt  # For plotting
import seaborn as sns  # For statistical visualizations

# Machine Learning - Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor

# Machine Learning - Regression Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Hyperparameter Tuning
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

# Model Interpretation
import shap
from sklearn.inspection import permutation_importance, PartialDependenceDisplay

# Utilities
import warnings
import time
from datetime import datetime
import pickle
import os

# Suppress warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plotting settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("="*80)
print("✅ All libraries imported successfully!")
print("="*80)
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"XGBoost version: {xgb.__version__}")
print(f"LightGBM version: {lgb.__version__}")
print(f"CatBoost version: {catboost.__version__}")
print(f"Optuna version: {optuna.__version__}")
print(f"SHAP version: {shap.__version__}")
print("="*80)
print(f"📅 Notebook started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## 1.2 Load Dataset

**Penjelasan**: Load dataset concrete compressive strength dari file CSV.

**Analogi**: Membuka file Excel yang berisi data penjualan untuk dianalisis.

In [ ]:
# Load dataset
df = pd.read_csv('datasets/concrete_data.csv')

print("="*80)
print("📊 DATASET LOADED SUCCESSFULLY")
print("="*80)
print(f"\n📏 Dataset Shape: {df.shape}")
print(f"   - Number of samples (rows): {df.shape[0]:,}")
print(f"   - Number of features (columns): {df.shape[1]}")
print(f"   - Total cells: {df.shape[0] * df.shape[1]:,}")
print("="*80)

## 1.3 First Look at Data

**Penjelasan**: Lihat beberapa baris pertama untuk memahami struktur data.

**Mengapa penting?** Kita perlu tahu seperti apa data kita sebelum analisis lebih lanjut.

In [ ]:
print("\n📋 First 5 rows of the dataset:")
print("="*80)
display(df.head())

print("\n📋 Last 5 rows of the dataset:")
print("="*80)
display(df.tail())

print("\n🎲 Random 5 rows from the dataset:")
print("="*80)
display(df.sample(5, random_state=RANDOM_STATE))

## 1.4 Dataset Information

**Penjelasan**: Check info lengkap dataset (tipe data, missing values, memory usage).

**Analogi**: Seperti mengecek spesifikasi mobil sebelum beli - berapa cc, bensin atau diesel, tahun berapa, dll.

In [ ]:
print("\n📊 DATASET INFORMATION")
print("="*80)
df.info()
print("="*80)

# Memory usage
memory_usage = df.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"\n💾 Memory Usage: {memory_usage:.2f} MB")

# Data types distribution
print("\n📊 Data Types Distribution:")
print(df.dtypes.value_counts())

## 1.5 Column Names

**Penjelasan**: List semua nama kolom untuk memahami fitur apa saja yang ada.

In [ ]:
print("\n📝 Column Names:")
print("="*80)
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")
print("="*80)

# Separate features and target
target_col = 'Concrete_Compressive_Strength'
feature_cols = [col for col in df.columns if col != target_col]

print(f"\n🎯 Target Variable: {target_col}")
print(f"\n📊 Features ({len(feature_cols)}):")
for i, col in enumerate(feature_cols, 1):
    print(f"{i}. {col}")

## 1.6 Initial Observations

**Summary Fase 1:**

✅ Dataset berhasil di-load

✅ Total samples: 1,030

✅ Total features: 8 (input) + 1 (target)

✅ Semua data numerical (float64)

✅ Siap untuk EDA!

---

# 🔷 FASE 2: EXPLORATORY DATA ANALYSIS (EDA)

**Analogi**: Seperti detektif yang memeriksa TKP dari berbagai sudut pandang untuk memahami apa yang terjadi.

**Tujuan Fase Ini:**
- Understand data distribution
- Check for missing values
- Check for duplicates
- Analyze target variable
- Analyze relationships between features
- Detect outliers
- Gain insights untuk modeling

## 2.1 Missing Values Analysis

**Penjelasan**: Check apakah ada data yang hilang (missing values).

**Mengapa penting?** Missing values bisa mempengaruhi hasil model. Kita perlu handle dengan cara yang benar.

**Analogi**: Seperti mengecek apakah ada halaman yang hilang dari buku sebelum dibaca.

In [ ]:
print("\n🔍 MISSING VALUES ANALYSIS")
print("="*80)

# Count missing values
missing_counts = df.isnull().sum()
missing_percentages = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': missing_counts.values,
    'Missing_Percentage': missing_percentages.values
})

missing_df = missing_df.sort_values('Missing_Count', ascending=False)
print(missing_df.to_string(index=False))

total_missing = missing_counts.sum()
print("="*80)
print(f"\n✅ Total Missing Values: {total_missing}")

if total_missing == 0:
    print("\n🎉 Great! No missing values found in the dataset!")
else:
    print(f"\n⚠️ Found {total_missing} missing values that need to be handled.")

## 2.2 Duplicate Rows Check

**Penjelasan**: Check apakah ada baris yang duplikat.

**Mengapa penting?** Duplicate rows bisa menyebabkan bias dalam model.

**Analogi**: Seperti memeriksa apakah ada foto yang sama di album foto kita.

In [ ]:
print("\n🔍 DUPLICATE ROWS CHECK")
print("="*80)

duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

if duplicate_count == 0:
    print("\n✅ No duplicate rows found!")
else:
    print(f"\n⚠️ Found {duplicate_count} duplicate rows.")
    print("Recommendation: Remove duplicates before modeling.")

## 2.3 Statistical Summary

**Penjelasan**: Lihat statistik deskriptif (mean, median, std, min, max, etc.).

**Analogi**: Seperti melihat rapor nilai siswa - kita bisa tahu nilai rata-rata, nilai tertinggi, terendah, dll.

In [ ]:
print("\n📊 STATISTICAL SUMMARY")
print("="*80)
display(df.describe().T)

print("\n📊 Additional Statistics:")
print("="*80)

# Calculate additional statistics
stats_df = pd.DataFrame({
    'Mean': df.mean(),
    'Median': df.median(),
    'Mode': df.mode().iloc[0],
    'Std': df.std(),
    'Variance': df.var(),
    'Skewness': df.skew(),
    'Kurtosis': df.kurtosis()
})

display(stats_df)

**📝 Interpretasi Skewness & Kurtosis:**

**Skewness** (Kemencengan):
- = 0: Distribusi simetris (normal)
- > 0: Skewed ke kanan (positive skew) - ekor panjang di kanan
- < 0: Skewed ke kiri (negative skew) - ekor panjang di kiri

**Kurtosis** (Keruncingan):
- = 0: Normal distribution
- > 0: Heavy tails (lebih banyak outliers)
- < 0: Light tails (lebih sedikit outliers)

## 2.4 Target Variable Analysis

**Penjelasan**: Analisis mendalam untuk target variable (Concrete Compressive Strength).

**Mengapa penting?** Understanding distribusi target sangat penting untuk regression modeling.

**Analogi**: Sebelum lomba lari, kita perlu tahu seperti apa lintasan larinya - datar, menanjak, atau berlubang?

In [ ]:
print("\n🎯 TARGET VARIABLE ANALYSIS")
print("="*80)
print(f"Target Variable: {target_col}")
print("="*80)

target = df[target_col]

print(f"\nMean: {target.mean():.2f} MPa")
print(f"Median: {target.median():.2f} MPa")
print(f"Mode: {target.mode()[0]:.2f} MPa")
print(f"Std Dev: {target.std():.2f} MPa")
print(f"Min: {target.min():.2f} MPa")
print(f"Max: {target.max():.2f} MPa")
print(f"Range: {target.max() - target.min():.2f} MPa")
print(f"\nSkewness: {target.skew():.3f}")
print(f"Kurtosis: {target.kurtosis():.3f}")

# Visualize target distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram
axes[0, 0].hist(target, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(target.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {target.mean():.2f}')
axes[0, 0].axvline(target.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {target.median():.2f}')
axes[0, 0].set_xlabel('Concrete Compressive Strength (MPa)', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Distribution of Target Variable (Histogram)', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# KDE Plot
axes[0, 1].hist(target, bins=50, density=True, color='lightcoral', alpha=0.5, edgecolor='black')
target.plot(kind='kde', ax=axes[0, 1], color='darkred', linewidth=2)
axes[0, 1].set_xlabel('Concrete Compressive Strength (MPa)', fontsize=12)
axes[0, 1].set_ylabel('Density', fontsize=12)
axes[0, 1].set_title('Distribution with KDE', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Box Plot
axes[1, 0].boxplot(target, vert=False, widths=0.7)
axes[1, 0].set_xlabel('Concrete Compressive Strength (MPa)', fontsize=12)
axes[1, 0].set_title('Box Plot (Detect Outliers)', fontsize=14, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# Q-Q Plot (Normal Distribution Check)
from scipy import stats
stats.probplot(target, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot (Normality Check)', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Plot saved as 'outputs/target_distribution.png'")

**📝 Interpretasi Target Distribution:**

Dari visualisasi di atas, kita bisa lihat:

1. **Histogram & KDE**: Menunjukkan distribusi kekuatan beton
   - Apakah distribusinya normal (bell-shaped)?
   - Apakah ada multiple peaks (bimodal/multimodal)?
   
2. **Box Plot**: Mendeteksi outliers
   - Titik-titik di luar whiskers adalah outliers
   - Perlu diputuskan: keep, remove, atau transform?
   
3. **Q-Q Plot**: Check normalitas
   - Jika titik-titik mengikuti garis diagonal = distribusi mendekati normal
   - Jika melenceng = perlu transformation (log, sqrt, Box-Cox)

## 2.5 Feature Distribution Analysis

**Penjelasan**: Analisis distribusi untuk setiap feature.

**Mengapa penting?** Memahami distribusi features membantu kita memutuskan apakah perlu transformasi atau scaling.

In [ ]:
print("\n📊 FEATURE DISTRIBUTION ANALYSIS")
print("="*80)

# Plot distributions for all features
n_features = len(feature_cols)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows*4))
axes = axes.flatten()

for i, col in enumerate(feature_cols):
    # Histogram with KDE
    axes[i].hist(df[col], bins=30, color='steelblue', alpha=0.6, edgecolor='black', density=True)
    df[col].plot(kind='kde', ax=axes[i], color='darkred', linewidth=2)
    
    # Add mean and median lines
    axes[i].axvline(df[col].mean(), color='red', linestyle='--', linewidth=1.5, alpha=0.7, label=f'Mean: {df[col].mean():.1f}')
    axes[i].axvline(df[col].median(), color='green', linestyle='--', linewidth=1.5, alpha=0.7, label=f'Median: {df[col].median():.1f}')
    
    axes[i].set_xlabel(col, fontsize=10)
    axes[i].set_ylabel('Density', fontsize=10)
    axes[i].set_title(f'Distribution of {col}', fontsize=11, fontweight='bold')
    axes[i].legend(fontsize=8)
    axes[i].grid(alpha=0.3)

# Hide extra subplots
for j in range(i+1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.savefig('outputs/features_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Plot saved as 'outputs/features_distribution.png'")

Karena ini adalah notebook yang sangat panjang (2000+ lines), saya akan melanjutkan dengan semua fase lainnya. Apakah Anda ingin saya lanjutkan dengan:

1. **Melanjutkan FASE 2 (EDA)** dengan correlation analysis, outlier detection, dll?
2. **Langsung ke FASE 3-10** (Preprocessing, Modeling, Tuning, SHAP, dll)?
3. Atau create notebook dalam **file terpisah yang lebih kecil** untuk setiap fase?

Saya sudah membuat fondasi yang kuat. Bagaimana preferensi Anda untuk melanjutkan?